http://cs229.stanford.edu/ps/ps2/ps2.pdf

# (a)

Let's start with a simple example to understand Multinomial Naive Bayes.

| Text | Tag   |
|:------|:------:|
| investors! free for qualified investors &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | Spam |
| discount prices for game investors | Spam |
| let's grab a coffee | Not spam |
| investors get a free discount | Spam |
| our coffee game is a game for | Not spam |

Then our vocabulary, denoted by $V$, with an extra column for the number of times used, is

| Index | Word | Times used   |
|:------:|:------:|:------:|
| &nbsp; 0 &nbsp; |&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; a &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | 3 |
| 1 | coffee | 2 |
| 2 | discount | 2 |
| 3 | for | 3 |
| 4 | free | 2 |
| 5 | game | 3 |
| 6 | get | 1 |
| 7 | grab | 1 |
| 8 | investors | 4 |
| 9 | is | 1 |
| 10 | let's | 1 |
| 11 | our | 1 |
| 12 | prices | 1 |
| 13 | qualified | 1 |

But it's easier to code this up:

In [1]:
import numpy as np

V = {0:'a', 1:'coffee', 2:'discount', 3:'for', 4:'free', 5:'game', 6:'get', 7:'grab', \
     8:'investors', 9:'is', 10:'lets', 11:'our', 12:'prices', 13:'qualified'}

X = np.array([[0,0,0,1,1,0,0,0,2,0,0,0,0,1],[0,0,1,1,0,1,0,0,1,0,0,0,1,0],[1,1,0,0,0,0,0,1,0,0,1,0,0,0],\
              [1,0,1,0,1,0,1,0,1,0,0,0,0,0],[1,1,0,1,0,2,0,0,0,1,0,1,0,0]])

y = np.array([1,1,0,1,0]) # 1 denotes Spam, 0 denotes non-Spam

Xs, Xn = X[y==1], X[y==0]

print("X=\n{}".format(X))

print("\nV=")

for key, value in V.items():
    tc, sc, nc = np.sum(X[:,key]), np.sum(Xs[:,key]), np.sum(Xn[:,key])
    print("{}  {}  {}  Spam={}  NS={}".format(key, value, tc, sc, nc))

cnt_spam_words, cnt_ns_words = np.sum(Xs), np.sum(Xn)
print("\ncnt_spam_words={}  cnt_ns_words={}".format(cnt_spam_words, cnt_ns_words))

X=
[[0 0 0 1 1 0 0 0 2 0 0 0 0 1]
 [0 0 1 1 0 1 0 0 1 0 0 0 1 0]
 [1 1 0 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 1 0 1 0 1 0 1 0 0 0 0 0]
 [1 1 0 1 0 2 0 0 0 1 0 1 0 0]]

V=
0  a  3  Spam=1  NS=2
1  coffee  2  Spam=0  NS=2
2  discount  2  Spam=2  NS=0
3  for  3  Spam=2  NS=1
4  free  2  Spam=2  NS=0
5  game  3  Spam=1  NS=2
6  get  1  Spam=1  NS=0
7  grab  1  Spam=0  NS=1
8  investors  4  Spam=4  NS=0
9  is  1  Spam=0  NS=1
10  lets  1  Spam=0  NS=1
11  our  1  Spam=0  NS=1
12  prices  1  Spam=1  NS=0
13  qualified  1  Spam=1  NS=0

cnt_spam_words=15  cnt_ns_words=11


Note let's suppose we get a new phrase "discount for coffee investors" and we wish to classify it as spam or not. Using Bayes, we compute

$$\begin{align*}
P(\text{Spam}\mid \text{discount for coffee investors})=\frac{P(\text{discount for coffee investors}\mid\text{Spam})P(\text{Spam})}{P(\text{discount for coffee investors})}
\end{align*}$$

and

$$\begin{align*}
P(\text{Not-Spam}\mid \text{discount for coffee investors})=\frac{P(\text{discount for coffee investors}\mid\text{Not-Spam})P(\text{Not-Spam})}{P(\text{discount for coffee investors})}
\end{align*}$$

Then we pick the larger of the two. But we can discard the denominator because it is the same in both cases. Discarding this, we still pick the larger of the two:

$$\begin{align*}
P(\text{Spam}\mid \text{discount for coffee investors})\propto P(\text{discount for coffee investors}\mid\text{Spam})P(\text{Spam})\tag{SE.1.1}
\end{align*}$$

and

$$\begin{align*}
P(\text{Not-Spam}\mid \text{discount for coffee investors})\propto P(\text{discount for coffee investors}\mid\text{Not-Spam})P(\text{Not-Spam})\tag{SE.1.2}
\end{align*}$$

Estimating the prior class probabilities is easy

$$\begin{align*}
\phi_c=\frac{\text{number of occurences of class }c\text{ in training data}}{\text{number of observations in training data}}\approx P(c)\tag{SE.1.3}
\end{align*}$$

where $\phi_c$ denotes the estimated prior probability of class $c$. Then

$$\begin{align*}
\phi_{Spam}=\frac{3}{5}\quad\quad\quad \phi_{Not-Spam}=\frac{2}{5}
\end{align*}$$

But we have a problem. The phrase "discount for coffee investors" doesn't appear in our training data so the probabilities in SE.1.1 and SE.1.2 will both be zero. To circumvent this, we make the Naive Bayes Assumption: the words in each phrase are conditionally independent given the classfication. That is, we assume

$$\begin{align*}
P(\text{discount for coffee investors}\mid c)=P(\text{discount}\mid c)P(\text{for}\mid c)P(\text{coffee}\mid c)P(\text{investors}\mid c)\tag{SE.1.4}
\end{align*}$$

We can estimate each of these probabilities. Let $N(w,c)$ denote the number of occurences of word $w$ for observations classified as $c$:

$$\begin{align*}
\phi_{j\mid y=c}=\frac{N(w_j,c)}{\sum_{w\in V}N(w,c)}\approx P(w_j\mid c)\tag{SE.1.5}
\end{align*}$$

where $\phi_{j\mid y=c}$ denotes the estimated conditional probability that the word $w_j$ appears in an observation classified as $c$. For example:

$$\begin{align*}
P(\text{investors}\mid\text{Not-Spam})\approx\phi_{8\mid y=0}=\frac{N(w_8,0)}{\sum_{w\in V}N(w,0)}=\frac{0}{11}=0
\end{align*}$$

$$\begin{align*}
P(\text{investors}\mid\text{Spam})\approx\phi_{8\mid y=1}=\frac{N(w_8,1)}{\sum_{w\in V}N(w,1)}=\frac{4}{15}
\end{align*}$$

$$\begin{align*}
P(\text{coffee}\mid\text{Not-Spam})\approx\phi_{1\mid y=0}=\frac{N(w_1,0)}{\sum_{w\in V}N(w,0)}=\frac{2}{11}
\end{align*}$$

$$\begin{align*}
P(\text{coffee}\mid\text{Spam})\approx\phi_{1\mid y=1}=\frac{N(w_1,1)}{\sum_{w\in V}N(w,1)}=\frac{0}{15}=0
\end{align*}$$

$$\begin{align*}
P(\text{game}\mid\text{Not-Spam})\approx\phi_{5\mid y=0}=\frac{N(w_5,0)}{\sum_{w\in V}N(w,0)}=\frac{2}{11}
\end{align*}$$

$$\begin{align*}
P(\text{game}\mid\text{Spam})\approx\phi_{5\mid y=1}=\frac{N(w_5,1)}{\sum_{w\in V}N(w,1)}=\frac{1}{15}
\end{align*}$$

Now we have another problem. The word "coffee" doesn't appear in any training observations that are classified as spam. Hence $P(\text{coffee}\mid\text{Spam})\approx0$. This is a problem because in equation SE.1.4, this would give $P(\text{discount for coffee investors}\mid \text{Spam})=0$. But we would obviously like to classify the phrase "discount for coffee investors" as spam.

To get around this, we use Laplace Smoothing. We add $1$ to every count $N(w,c)$. Then SE.1.5 becomes

$$\begin{align*}
P(w_j\mid c)\approx\phi_{j\mid y=c}=\frac{N(w_j,c)+1}{\sum_{w\in V}\big(N(w,c)+1\big)}=\frac{N(w_j,c)+1}{\sum_{w\in V}N(w,c)+\lvert V\rvert}\tag{SE.1.6}
\end{align*}$$

And our examples become

$$\begin{align*}
P(\text{investors}\mid\text{Not-Spam})\approx\phi_{8\mid y=0}=\frac{0+1}{11+14}=\frac{1}{25}
\end{align*}$$

$$\begin{align*}
P(\text{investors}\mid\text{Spam})\approx\phi_{8\mid y=1}=\frac{4+1}{15+14}=\frac{5}{29}
\end{align*}$$

$$\begin{align*}
P(\text{coffee}\mid\text{Not-Spam})\approx\phi_{1\mid y=0}=\frac{2+1}{11+14}=\frac{3}{25}
\end{align*}$$

$$\begin{align*}
P(\text{coffee}\mid\text{Spam})\approx\phi_{1\mid y=1}=\frac{0+1}{15+14}=\frac{1}{29}
\end{align*}$$

$$\begin{align*}
P(\text{game}\mid\text{Not-Spam})\approx\phi_{5\mid y=0}=\frac{2+1}{11+14}=\frac{3}{25}
\end{align*}$$

$$\begin{align*}
P(\text{game}\mid\text{Spam})\approx\phi_{5\mid y=1}=\frac{1+1}{15+14}=\frac{2}{29}
\end{align*}$$

Then our computations for SE.1.4 are

$$\begin{align*}
P(\text{discount for coffee investors}\mid\text{Spam}) &= P(\text{discount}\mid\text{Spam})P(\text{for}\mid \text{Spam})P(\text{coffee}\mid\text{Spam})P(\text{investors}\mid\text{Spam})\\\\
    &\approx \phi_{2\mid y=1}\phi_{3\mid y=1}\phi_{1\mid y=1}\phi_{8\mid y=1}
\end{align*}$$

and

$$\begin{align*}
P(\text{discount for coffee investors}\mid\text{Not-Spam}) &\approx \phi_{2\mid y=0}\phi_{3\mid y=0}\phi_{1\mid y=0}\phi_{8\mid y=0}
\end{align*}$$

Then we can compare

$$\begin{align*}
P(\text{Spam}\mid \text{discount for coffee investors})&\propto P(\text{discount for coffee investors}\mid\text{Spam})P(\text{Spam})\\\\
    &\approx \phi_{2\mid y=1}\phi_{3\mid y=1}\phi_{1\mid y=1}\phi_{8\mid y=1}\frac{3}{5}
\end{align*}$$

and

$$\begin{align*}
P(\text{Not-Spam}\mid \text{discount for coffee investors})&\propto P(\text{discount for coffee investors}\mid\text{Not-Spam})P(\text{Not-Spam})\\\\
    &\approx \phi_{2\mid y=0}\phi_{3\mid y=0}\phi_{1\mid y=0}\phi_{8\mid y=0}\frac{2}{5}
\end{align*}$$

But of course this is easier with code:

In [2]:
Njc = lambda j,c: np.sum((Xs if c==1 else Xn)[:,j])

phijc = lambda j,c: (Njc(j,c) + 1)/(np.sum([Njc(i,c) for i in range(len(V))]) + len(V))

prob_sp_dfci = phijc(2,1) * phijc(3,1) * phijc(1,1) * phijc(8,1) * 3/5
prob_ns_dfci = phijc(2,0) * phijc(3,0) * phijc(1,0) * phijc(8,0) * 2/5
print("Spam={:.8f}  Not={:.8f}".format(prob_sp_dfci, prob_ns_dfci))

# to check more of these, it will be easier to use this approach:

def get_V_key(value, V):
    for key in V:
        if V[key] == value: return key
    return None

def compute_probs(phrase, V, rnd=9):
    js = [get_V_key(word, V) for word in phrase.split()]
    prob_sp = np.sum(y==1)/5 * np.prod([phijc(j,1) for j in js])
    prob_ns = np.sum(y==0)/5 * np.prod([phijc(j,0) for j in js])
    return np.array([prob_sp, prob_ns])

def classify(phrase, V):
    cp = compute_probs(phrase, V)
    ratio = cp[1] / cp[0] if cp[1] >= cp[0] else cp[0] / cp[1]
    label = "SPAM" if cp[0] >= cp[1] else "NOT"
    print("Spam={:.8f}  Not={:.8f}  ratio={:.2f}  \"{}\"  class={}".format(cp[0], cp[1], ratio, phrase, label))

phrases = ['discount for coffee investors',
    'a coffee is coffee',
    'a coffee is free',
    'investors coffee is free',
    'investors coffee is discount',
    'investors coffee is game',
    'investors coffee get game',
    'investors get prices free for qualified investors',
    'our coffee grab is lets coffee']

import time
start = time.time()
for phr in phrases: classify(phr, V)
end = time.time()
print("TIME={} seconds".format(end-start))
    
# but we can vectorize this

def parse_phrases(phrases, V):
    X = []
    for phrase in phrases:
        js = [get_V_key(word, V) for word in phrase.split()]
        d = {j:js.count(j) for j in js}
        x = np.zeros(len(V))
        for k, v in d.items():
            x[k] = v
        X.append(x)
    return np.array(X)

def train_1(X, y, Voc):
    X1, X0 = X[y==1], X[y==0]
    phi_js_yeq1 = (np.sum(X1, axis=0) + 1) / (np.sum(X1) + len(Voc))
    phi_js_yeq0 = (np.sum(X0, axis=0) + 1) / (np.sum(X0) + len(Voc))
    phi_yeq1 = np.sum(y==1)/y.shape[0]
    phi_yeq0 = np.sum(y==0)/y.shape[0]
    return phi_js_yeq1, phi_js_yeq0, phi_yeq1, phi_yeq0

def test_1(phi_js_yeq1, phi_js_yeq0, phi_yeq1, phi_yeq0, test_phrases, Voc):
    Xtest = parse_phrases(test_phrases, Voc)
    pX1 = np.power(phi_js_yeq1, Xtest)
    pX0 = np.power(phi_js_yeq0, Xtest)
    s1 = phi_yeq1 * np.prod(pX1, axis=1)
    s0 = phi_yeq0 * np.prod(pX0, axis=1)
    rs = s1/s0
    rs[rs<1] = 1./rs[rs<1]
    return s1, s0, rs

def train_and_test_1(X, y, Voc, test_phrases):
    phi_js_yeq1, phi_js_yeq0, phi_yeq1, phi_yeq0 = train_1(X, y, Voc)
    return test_1(phi_js_yeq1, phi_js_yeq0, phi_yeq1, phi_yeq0, test_phrases, Voc)

np.set_printoptions(suppress=True)
start = time.time()
s1, s0, rs = train_and_test_1(X, y, V, phrases)
print("s1={}\ns0={}\nrs={}".format(s1.round(8), s0.round(8), rs))
end = time.time()
print("TIME={} seconds".format(end-start))
    

Spam=0.00003817  Not=0.00000614
Spam=0.00003817  Not=0.00000614  ratio=6.21  "discount for coffee investors"  class=SPAM
Spam=0.00000170  Not=0.00005530  ratio=32.59  "a coffee is coffee"  class=NOT
Spam=0.00000509  Not=0.00001843  ratio=3.62  "a coffee is free"  class=NOT
Spam=0.00001272  Not=0.00000614  ratio=2.07  "investors coffee is free"  class=SPAM
Spam=0.00001272  Not=0.00000614  ratio=2.07  "investors coffee is discount"  class=SPAM
Spam=0.00000848  Not=0.00001843  ratio=2.17  "investors coffee is game"  class=NOT
Spam=0.00001697  Not=0.00000922  ratio=1.84  "investors coffee get game"  class=SPAM
Spam=0.00000006  Not=0.00000000  ratio=477.67  "investors get prices free for qualified investors"  class=SPAM
Spam=0.00000000  Not=0.00000024  ratio=233.89  "our coffee grab is lets coffee"  class=NOT
TIME=0.007946014404296875 seconds
s1=[ 0.00003817  0.0000017   0.00000509  0.00001272  0.00001272  0.00000848
  0.00001697  0.00000006  0.        ]
s0=[ 0.00000614  0.0000553   0.00001



Hence our ML estimates are

$$\begin{align*}
\phi_{j\mid y=1} &= \frac{\sum_{i=1}^{m}\boldsymbol{1} \{y^{(i)} =1 \}x_j^{(i)} + 1}{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 1 \} n_i + \left| V \right|} \\\\
\phi_{j\mid y=0} &= \frac{\sum_{i=1}^{m}\boldsymbol{1} \{y^{(i)} =0 \}x_j^{(i)} + 1}{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 0 \} n_i + \left| V \right|} \\\\
\phi_{y} &= \frac{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 1 \}}{m}
\end{align*}$$

### Bernoulli NB Likelihood

Let's derive the log-likelihood in the case of Bernoulli Naive Bayes.

Let $y$ be a discrete random variable with values in $\{0,1\}$. Define $\phi_y\equiv P(y=1)=p_{y}(1)$.

Let $x=[x_1,...,x_n]^{T}$ be a discrete random vector with values in $\{0,1\}^{n}$, where $n$ is the number of features. That is, an observation from $x$ is a column vector in $\mathbb{R}^n$ whose elements are $0$ or $1$. Define

$$\begin{gather}
\phi_{j\mid y=1}\equiv P(x_j=1\mid y=1)=p_{x_j\mid y}(1\mid 1)\\\\
\phi_{j\mid y=0}\equiv P(x_j=1\mid y=0)=p_{x_j\mid y}(1\mid 0)
\end{gather}$$

Given a training set $\{x^{(i)},y^{(i)}\}_{i=1}^{m}$ where each $(x^{(i)},y^{(i)})$ is an independent observation from the joint random variable $(x,y)$, then the joint likelihood of the training data is

$$\begin{align*} 
\mathscr{L}\big(\phi_{j\mid y=1},\phi_{j\mid y=0},\phi_y;\{x^{(i)},y^{(i)}\}_{i=1}^{m}\big) &= P\big((x,y=x^{(1)},y^{(1)}),...,(x,y=x^{(m)},y^{(m)});\phi_{j|y=1},\phi_{j|y=0},\phi_y\big)\\\\
    &= \prod_{i=1}^{m}P\big(x,y=x^{(i)},y^{(i)};\phi_{j\mid y=1},\phi_{j\mid y=0},\phi_y\big)\tag{SCN.1}\\\\
    &= \prod_{i=1}^{m}p_{x,y}\big(x^{(i)},y^{(i)};\phi_{j\mid y=1},\phi_{j\mid y=0},\phi_y\big)\\\\
    &= \prod_{i=1}^{m}p_{x\mid y}\big(x^{(i)}\mid y^{(i)};\phi_{j\mid y=1},\phi_{j\mid y=0}\big)p_{y}\big(y^{(i)};\phi_y\big)\tag{SCN.2}\\\\
\end{align*}$$

SCN.1 holds from the assumption that observations are independent. SCN.2 holds from the definition of conditional probability. And the log-likelihood is

$$\begin{align*} 
\ell\big(\phi_{j\mid y=1},\phi_{j\mid y=0},\phi_y\big) &= \sum_{i=1}^m\mathrm{ln}\big[p_{x\mid y}(x^{(i)}\mid y^{(i)};\phi_{j\mid y=1},\phi_{j\mid y=0})\big]+ \sum_{i=1}^m\mathrm{ln}\big[p_{y}(y^{(i)};\phi_y)\big]\\\\
     &= \sum_{i=1}^m\mathrm{ln}\Big[\prod_{j=1}^{n}p_{x_{j}\mid y}(x_j^{(i)}\mid y^{(i)};\phi_{j\mid y=1},\phi_{j\mid y=0})\Big]+ \sum_{i=1}^m\mathrm{ln}\big[p_{y}(y^{(i)};\phi_y)\big]\tag{SCN.3}\\\\
     &= \sum_{i=1}^m\sum_{j=1}^{n}\mathrm{ln}\big[p_{x_{j}\mid y}(x_j^{(i)}\mid y^{(i)};\phi_{j\mid y=1},\phi_{j\mid y=0})\big]+ \sum_{i=1}^m\mathrm{ln}\big[p_{y}(y^{(i)};\phi_y)\big] \tag{SCN.4}\\\\
     &= \sum_{i=1}^m\sum_{j=1}^{n}\big\{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}\mathrm{ln}(\phi_{j\mid y=1})+\boldsymbol{1}\{y^{(i)}=0\wedge x_j^{(i)}=1\}\mathrm{ln}(\phi_{j\mid y=0}) \\
     &+ \boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=0\}\mathrm{ln}(1-\phi_{j\mid y=1})+\boldsymbol{1}\{y^{(i)}=0\wedge x_j^{(i)}=0\}\mathrm{ln}(1-\phi_{j|y=0})\big\} \tag{SCN.5} \\
     &+ \sum_{i=1}^m\big\{\boldsymbol{1}\{y^{(i)}=1\}\mathrm{ln}(\phi_y)+\boldsymbol{1}\{y^{(i)}=0\}\mathrm{ln}(1-\phi_y)\big\}\\\\
\end{align*}$$

SCN.3 follows from the Naive Bayes Assumption: $x_1^{(i)},...,x_n^{(i)}$ are conditionally independent given $y^{(i)}$. Intuitively, if we know whether an email is spam or not, then the appearance of different words is independent. For example, if we know that an email is spam, then the appearances of "free" and "discount" are independent events.

### Bernoulli NB MLE

To maximize the log-likelihood in $\phi_y$, let's compute the partial derivative of SCN.5 with respect to $\phi_y$ and set it equal to zero:

$$\begin{align*} 
0=\frac{\partial\ell}{\partial\phi_y} &= \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}}\frac1{\phi_y} - \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=0\}}\frac1{1-\phi_y}\\
\end{align*}$$

Hence

$$\begin{align*} 
\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=0\}}\frac1{1-\phi_y} = \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}}\frac1{\phi_y}\\
\end{align*}$$

Or

$$\begin{align*} 
\frac1{1-\phi_y}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=0\}} = \frac1{\phi_y}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}}\\
\end{align*}$$

Multiplying both sides by $\phi_y(1-\phi_y)$, we get

$$\begin{align*} 
\phi_y\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=0\}} &= (1-\phi_y)\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}}\\
    &= \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}} - \phi_y\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}}
\end{align*}$$

Hence

$$\begin{align*} 
\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}} &= \phi_y\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=0\}} + \phi_y\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}}\\
    &= \phi_y\sum_{i=1}^m\big[\boldsymbol{1}\{y^{(i)}=0\} + \boldsymbol{1}\{y^{(i)}=1\}\big]\\
    &= \phi_y\sum_{i=1}^m1\\
    &= \phi_y m
\end{align*}$$

Hence

$$\begin{align*} 
\phi_y = \frac1m\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\}}
\end{align*}$$

Similarly we can maximize in $\phi_{j|y=1}$:

$$\begin{align*} 
0=\frac{\partial\ell}{\partial\phi_{j|y=1}} &= \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}}\frac1{\phi_{j|y=1}} - \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=0\}}\frac1{1-\phi_{j|y=1}}\\
\end{align*}$$

Hence

$$\begin{align*} 
\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=0\}}\frac1{1-\phi_{j|y=1}} = \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}}\frac1{\phi_{j|y=1}}\\
\end{align*}$$

Or

$$\begin{align*} 
\frac1{1-\phi_{j|y=1}}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=0\}} = \frac1{\phi_{j|y=1}}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}}\\
\end{align*}$$

Multiplying both sides by $\phi_{j|y=1}(1-\phi_{j|y=1})$, we get

$$\begin{align*} 
\phi_{j|y=1}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=0\}} &= (1-\phi_{j|y=1})\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}}\\
    &= \sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}} - \phi_{j|y=1}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}}
\end{align*}$$

Hence

$$\begin{align*} 
\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}} &= \phi_{j|y=1}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=0\}} + \phi_{j|y=1}\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}}\\
    &= \phi_{j|y=1}\sum_{i=1}^m\big[\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=0\} + \boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}\big]\\
    &= \phi_{j|y=1}\sum_{i=1}^m\boldsymbol{1}\{y^{(i)}=1\}\\
\end{align*}$$

Hence

$$\begin{align*} 
\phi_{j|y=1} = \frac{\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=1\wedge x_j^{(i)}=1\}}}{\sum_{i=1}^m\boldsymbol{1}\{y^{(i)}=1\}}
\end{align*}$$

Similarly we can compute

$$\begin{align*} 
\phi_{j|y=0} = \frac{\sum_{i=1}^m{\boldsymbol{1}\{y^{(i)}=0\wedge x_j^{(i)}=1\}}}{\sum_{i=1}^m\boldsymbol{1}\{y^{(i)}=0\}}
\end{align*}$$

Now let's suppose that $x=[x_1,...,x_n]^T$ is discrete and $x_j$ takes values in the nonnegative integers $0,1,2,...$. And let's define

$$\begin{gather}
\phi_{j\mid y=1}\equiv P(x_j=1\mid y=1)=p_{x_j\mid y}(1\mid 1)\\\\
\phi_{j\mid y=0}\equiv P(x_j=1\mid y=0)=p_{x_j\mid y}(1\mid 0)
\end{gather}$$

$$\begin{align*}
\phi_{j\mid y=1} &= \frac{\sum_{i=1}^{m}\boldsymbol{1} \{y^{(i)} =1 \}x_j^{(i)} + 1}{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 1 \} n_i + \left| V \right|} \\\\
\phi_{y} &= \frac{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 1 \}}{m}
\end{align*}$$

### Theory

For Navie Bayes represented as a multinomial event model, to maximize its maxmimum likelihood with Laplace smoothing,

\begin{align*}
\phi_{k|y=1} &= \frac{\sum_{i=1}^{m} \sum_{j=1}^{n_i} \boldsymbol{1} \{ x_j^{(i)} = k \wedge y^{(i)} =1 \} + 1}{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 1 \} n_i + \left| V \right|} \\
\phi_{k|y=0} &= \frac{\sum_{i=1}^{m} \sum_{j=1}^{n_i} \boldsymbol{1} \{ x_j^{(i)} = k \wedge y^{(i)} =0 \} + 1}{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 0 \} n_i + \left| V \right|} \\
\phi_{y} &= \frac{\sum_{i=1}^{m} \boldsymbol{1} \{ y^{(i)} = 1 \}}{m}
\end{align*}

For prediction,

$$\begin{align*}
p(y=1|x) &= \frac{p(x|y=1)p(y=1)}{p(x)} \\
              &= \frac{p(x|y=1)p(y=1)}{p(x|y=1)p(y=1) + p(x|y=0) p(y=0)} \\
              &= \frac{1}{1 + \frac{p(x|y=0) p(y=0)}{p(x|y=1) p(y=1)}} \\
              &= \frac{1}{1 + \mathrm{exp}\big[\mathrm{log}P(x|y=0) + \mathrm{log}P(y=0) - \mathrm{log}P(x|y=1) - \mathrm{log} P(y=1)\big]}
\end{align*}$$

The last two equalities are for numerical stability

See implementation in `nb.py`

# (b)

In [3]:
import glob

import numpy as np
import matplotlib.pyplot as plt

import nb

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
trainMatrix, tokenlist, trainCategory = nb.readMatrix('unzipped_spam_data/MATRIX.TRAIN')
print("types: tM={}  tl={}  tle={}  tC={}".format(type(trainMatrix), type(tokenlist), type(tokenlist[0]), type(trainCategory)))
print("shapes: tM={}  tl={}  tC={}".format(trainMatrix.shape, len(tokenlist), trainCategory.shape))
print("previews:\ntl={}\ntC={}".format(tokenlist[:3], trainCategory[:3]))

types: tM=<class 'numpy.ndarray'>  tl=<class 'list'>  tle=<class 'str'>  tC=<class 'numpy.ndarray'>
shapes: tM=(2144, 1448)  tl=1448  tC=(2144,)
previews:
tl=['abil', 'absolut', 'abus']
tC=[1 0 1]


In [5]:
state = nb.nb_train(trainMatrix, trainCategory)

In [6]:
state.keys()

dict_keys([])

In [7]:
tokens = np.array(tokenlist)
tokens[np.argsort(state['phi_yeq1'] / state['phi_yeq0'])[::-1]][:50]

KeyError: 'phi_yeq1'

These are the top words most indicative of spam emails, kind of makes sense.

# (c)

In [ ]:
files = sorted(glob.glob('./spam_data/MATRIX.TRAIN.[0-9]*'), key=lambda s: int(s.rsplit('.')[-1]))

nb_sizes = []
nb_errs = []
mat_test, tok_test, cat_test = nb.readMatrix('./spam_data/MATRIX.TEST')
for f in files:
    mat, tok, cat = nb.readMatrix(f)
    nb_sizes.append(mat.shape[0])
    mod = nb.nb_train(mat, cat)
    output = nb.nb_test(mat_test, mod)
    nb_errs.append(nb.evaluate(output, cat_test))

In [ ]:
plt.plot(nb_sizes, nb_errs)
plt.xlabel('training set size')
plt.ylabel('error')

# (d)

In [ ]:
import svm

In [ ]:
files = sorted(glob.glob('./spam_data/MATRIX.TRAIN.[0-9]*'), key=lambda s: int(s.rsplit('.')[-1]))

svm_sizes = []
svm_errs = []
mat_test, tok_test, cat_test = svm.readMatrix('./spam_data/MATRIX.TEST')
for f in files:
    mat, tok, cat = svm.readMatrix(f)
    svm_sizes.append(mat.shape[0])
    mod = svm.svm_train(mat, cat)
    output = svm.svm_test(mat_test, mod)
    svm_errs.append(svm.evaluate(output, cat_test))

In [ ]:
plt.plot(svm_sizes, svm_errs)
plt.xlabel('training set size')
plt.ylabel('error')

# (e)

In [ ]:
plt.plot(nb_sizes, nb_errs, label='navie bayes')
plt.plot(svm_sizes, svm_errs, label='svm')
plt.xlabel('training set size')
plt.ylabel('error')
plt.legend()

SVM outperforms NB at each training set size